In [27]:
import numpy as np 
import requests
import json
import pandas as pd 

In [28]:
datos = requests.get('https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json')
resultado = json.loads(datos.text)
df = pd.DataFrame(resultado)

In [29]:
customer_df = pd.json_normalize(df['customer'])
phone_df    = pd.json_normalize(df['phone'])
internet_df = pd.json_normalize(df['internet'])
account_df  = pd.json_normalize(df['account'])

# Ahora concatenamos todo con las columnas originales que ya estaban planas
df = pd.concat(
    [df[['customerID', 'Churn']], customer_df, phone_df, internet_df, account_df],
    axis=1
)

In [30]:
df

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.9
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


In [31]:
df.columns = df.columns.str.strip()

In [32]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip()
    df[col] = df[col].replace('', np.nan)

col_yes_no = ['Churn', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
              'OnlineBackup','OnlineSecurity', 'DeviceProtection','TechSupport', 'StreamingTV',
              'StreamingMovies', 'PaperlessBilling']

for col in col_yes_no:
    print(f"\n--- {col} ---")
    print(df[col].value_counts(dropna=False))


--- Churn ---
Churn
No     5174
Yes    1869
NaN     224
Name: count, dtype: int64

--- Partner ---
Partner
No     3749
Yes    3518
Name: count, dtype: int64

--- Dependents ---
Dependents
No     5086
Yes    2181
Name: count, dtype: int64

--- PhoneService ---
PhoneService
Yes    6560
No      707
Name: count, dtype: int64

--- MultipleLines ---
MultipleLines
No                  3495
Yes                 3065
No phone service     707
Name: count, dtype: int64

--- OnlineBackup ---
OnlineBackup
No                     3182
Yes                    2504
No internet service    1581
Name: count, dtype: int64

--- OnlineSecurity ---
OnlineSecurity
No                     3608
Yes                    2078
No internet service    1581
Name: count, dtype: int64

--- DeviceProtection ---
DeviceProtection
No                     3195
Yes                    2491
No internet service    1581
Name: count, dtype: int64

--- TechSupport ---
TechSupport
No                     3582
Yes                    2104
No

In [33]:
df.isna().sum()

customerID            0
Churn               224
gender                0
SeniorCitizen         0
Partner               0
Dependents            0
tenure                0
PhoneService          0
MultipleLines         0
InternetService       0
OnlineSecurity        0
OnlineBackup          0
DeviceProtection      0
TechSupport           0
StreamingTV           0
StreamingMovies       0
Contract              0
PaperlessBilling      0
PaymentMethod         0
Charges.Monthly       0
Charges.Total        11
dtype: int64

In [34]:
df.dropna(inplace=True)

In [35]:
df.isna().sum()

customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Charges.Monthly     0
Charges.Total       0
dtype: int64

In [36]:
df['Charges.Total'] = df['Charges.Total'].astype('float64')

In [37]:
columnas_español = {
    'Churn': 'Abandono',
    'gender': 'Genero',
    'SeniorCitizen': 'Es_mayor_de_edad',
    'Partner': 'Pareja',
    'Dependents': 'Dependientes',
    'tenure': 'Meses_contrato',
    'PhoneService': 'Servicio_telefonico',
    'MultipleLines': 'Lineas_telefonicas_multiples',
    'InternetService': 'Tipo_servicio_internet',
    'OnlineSecurity': 'Seguridad_en_linea',
    'OnlineBackup': 'Respaldo_en_linea',
    'DeviceProtection': 'Proteccion_dispositivo',
    'TechSupport': 'Soporte_tecnico',
    'StreamingTV': 'TV_en_streaming',
    'StreamingMovies': 'Peliculas_en_streaming',
    'Contract': 'Tipo_contrato',
    'PaperlessBilling': 'Factura_en_linea',
    'PaymentMethod': 'Metodo_pago',
    'Charges.Monthly': 'Cargo_mensual',
    'Charges.Total': 'Cargos_totales'
}


df.rename(columns=columnas_español, inplace=True)
df.columns

Index(['customerID', 'Abandono', 'Genero', 'Es_mayor_de_edad', 'Pareja',
       'Dependientes', 'Meses_contrato', 'Servicio_telefonico',
       'Lineas_telefonicas_multiples', 'Tipo_servicio_internet',
       'Seguridad_en_linea', 'Respaldo_en_linea', 'Proteccion_dispositivo',
       'Soporte_tecnico', 'TV_en_streaming', 'Peliculas_en_streaming',
       'Tipo_contrato', 'Factura_en_linea', 'Metodo_pago', 'Cargo_mensual',
       'Cargos_totales'],
      dtype='object')

In [38]:
col_si_no = ['Abandono','Pareja', 'Dependientes','Servicio_telefonico', 
              'Lineas_telefonicas_multiples', 'Respaldo_en_linea', 
              'Seguridad_en_linea', 'Proteccion_dispositivo', 'Soporte_tecnico', 
              'TV_en_streaming', 'Peliculas_en_streaming', 'Factura_en_linea']
    
for col in df[col_si_no]:
    df[col] = df[col].map({'Yes': 1, 'No': 0})
    df[col] = df[col].fillna(0).astype(int)

df['Tipo_servicio_internet'] = df['Tipo_servicio_internet'].map({'Fiber optic': 'Fibra Optica', 'DSL': 'DSL', 'No': 'Sin servicio'})
df['Tipo_contrato'] = df['Tipo_contrato'].map({'Month-to-month':'Mes a mes','Two year':'Dos años','One year':'Un año'})
df['Metodo_pago'] = df['Metodo_pago'].map({'Electronic check':'Cheque electrónico', 'Mailed check':'Cheque por enviado por correo', 'Bank transfer (automatic)':'Transferencia bancaria (automática)', 'Credit card (automatic)':'Tarjeta de crédito (automática)'})


In [39]:
df['Cargos_diarios'] = df['Cargo_mensual']/30

In [40]:
df

,customerID,Abandono,Genero,Es_mayor_de_edad,Pareja,Dependientes,Meses_contrato,Servicio_telefonico,Lineas_telefonicas_multiples,Tipo_servicio_internet,...,Proteccion_dispositivo,Soporte_tecnico,TV_en_streaming,Peliculas_en_streaming,Tipo_contrato,Factura_en_linea,Metodo_pago,Cargo_mensual,Cargos_totales,Cargos_diarios
0,0002-ORFBO,0,Female,0,1,1,9,1,0,DSL,...,0,1,1,0,Un año,1,Cheque por enviado por correo,65.60,593.30,2.186667
1,0003-MKNFE,0,Male,0,0,0,9,1,1,DSL,...,0,0,0,1,Mes a mes,0,Cheque por enviado por correo,59.90,542.40,1.996667
2,0004-TLHLJ,1,Male,0,0,0,4,1,0,Fibra Optica,...,1,0,0,0,Mes a mes,1,Cheque electrónico,73.90,280.85,2.463333
3,0011-IGKFF,1,Male,1,1,0,13,1,0,Fibra Optica,...,1,0,1,1,Mes a mes,1,Cheque electrónico,98.00,1237.85,3.266667
4,0013-EXCHZ,1,Female,1,1,0,3,1,0,Fibra Optica,...,0,1,1,0,Mes a mes,1,Cheque por enviado por correo,83.90,267.40,2.796667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,0,Female,0,0,0,13,1,0,DSL,...,0,1,0,0,Un año,0,Cheque por enviado por correo,55.15,742.90,1.838333
7263,9992-RRAMN,1,Male,0,1,0,22,1,1,Fibra Optica,...,0,0,0,1,Mes a mes,1,Cheque electrónico,85.10,1873.70,2.836667
7264,9992-UJOEL,0,Male,0,0,0,2,1,0,DSL,...,0,0,0,0,Mes a mes,1,Cheque por enviado por correo,50.30,92.75,1.676667
7265,9993-LHIEB,0,Male,0,1,1,67,1,0,DSL,...,1,1,0,1,Dos años,0,Cheque por enviado por correo,67.85,4627.65,2.261667


In [41]:
df.value_counts('Tipo_servicio_internet')

Tipo_servicio_internet
Fibra Optica    3096
DSL             2416
Sin servicio    1520
Name: count, dtype: int64

In [42]:
df.columns

Index(['customerID', 'Abandono', 'Genero', 'Es_mayor_de_edad', 'Pareja',
       'Dependientes', 'Meses_contrato', 'Servicio_telefonico',
       'Lineas_telefonicas_multiples', 'Tipo_servicio_internet',
       'Seguridad_en_linea', 'Respaldo_en_linea', 'Proteccion_dispositivo',
       'Soporte_tecnico', 'TV_en_streaming', 'Peliculas_en_streaming',
       'Tipo_contrato', 'Factura_en_linea', 'Metodo_pago', 'Cargo_mensual',
       'Cargos_totales', 'Cargos_diarios'],
      dtype='object')

In [ ]:
# Calculate the total number of services purchased by each client
service_columns = ['Servicio_telefonico', 'Lineas_telefonicas_multiples', 'Respaldo_en_linea', 
                   'Seguridad_en_linea', 'Proteccion_dispositivo', 'Soporte_tecnico', 
                   'TV_en_streaming', 'Peliculas_en_streaming']

def calculate_services(row):
    total_services = row[service_columns].sum()
    if row['Tipo_servicio_internet'] == 'Sin servicio':
        return total_services
    else:
        return total_services + 1  # Add 1 for internet service if contracted

df['Total_servicios'] = df.apply(calculate_services, axis=1)

In [44]:
df.head()

,customerID,Abandono,Genero,Es_mayor_de_edad,Pareja,Dependientes,Meses_contrato,Servicio_telefonico,Lineas_telefonicas_multiples,Tipo_servicio_internet,...,Soporte_tecnico,TV_en_streaming,Peliculas_en_streaming,Tipo_contrato,Factura_en_linea,Metodo_pago,Cargo_mensual,Cargos_totales,Cargos_diarios,Total_servicios
0,0002-ORFBO,0,Female,0,1,1,9,1,0,DSL,...,1,1,0,Un año,1,Cheque por enviado por correo,65.6,593.30,2.186667,5
1,0003-MKNFE,0,Male,0,0,0,9,1,1,DSL,...,0,0,1,Mes a mes,0,Cheque por enviado por correo,59.9,542.40,1.996667,4
2,0004-TLHLJ,1,Male,0,0,0,4,1,0,Fibra Optica,...,0,0,0,Mes a mes,1,Cheque electrónico,73.9,280.85,2.463333,3
3,0011-IGKFF,1,Male,1,1,0,13,1,0,Fibra Optica,...,0,1,1,Mes a mes,1,Cheque electrónico,98.0,1237.85,3.266667,6
4,0013-EXCHZ,1,Female,1,1,0,3,1,0,Fibra Optica,...,1,1,0,Mes a mes,1,Cheque por enviado por correo,83.9,267.40,2.796667,4


In [47]:
df.shape[0]

7032

In [46]:
df.to_csv('Base_datos_tratada.csv', index= False)